In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

DATASET_PATH = '/content/drive/MyDrive/AudioWAV'
SAMPLE_RATE = 22050
DURATION = 2
NUM_CLASSES = 10


def extract_features(file_path, sample_rate, duration):
    try:
        audio, sr = librosa.load(file_path, sr=sample_rate, duration=duration)
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
        mfccs = np.mean(mfccs.T, axis=0)
        return mfccs
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None


def load_dataset(dataset_path, sample_rate, duration):
    features, labels = [], []
    for label, class_dir in enumerate(os.listdir(dataset_path)):
        class_path = os.path.join(dataset_path, class_dir)
        if os.path.isdir(class_path):
            for file in os.listdir(class_path):
                file_path = os.path.join(class_path, file)
                feature = extract_features(file_path, sample_rate, duration)
                if feature is not None:
                    features.append(feature)
                    labels.append(label)
    return np.array(features), np.array(labels)


print("Loading dataset...")
X, y = load_dataset(DATASET_PATH, SAMPLE_RATE, DURATION)
X = np.expand_dims(X, axis=0)
y = to_categorical(y, num_classes=NUM_CLASSES)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


print("Building model...")
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation="relu", input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(NUM_CLASSES, activation="softmax")
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


print("Training model...")
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)


print("Evaluating model...")
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


model.save("audio_classification_model.h5")
print("Model saved.")
